## shm - Shared Memory

I created this shared memory interface to pass variables amongst processes in linux for a detector firmware I was writing.  It uses the ``shmget`` and ``shmat`` C interface under the hood, but that's all so non-semantic when you code it that you shouldn't have to worry about it.

### Example

My fiance is watching the tv show Sherlock Holmes behind me, so this example is themed on that show.

First, we have to create some server for the shared memory.  This server has to be open for the client process to read it, so we'll just keep it open indefinitely.  We also have to define a unique identifier for the shared memory server.

In [1]:
from cppm import cppmagic

In [2]:
%%runcppnb shm_server
#include "../ah_shm.h"
#include "../ah_signal.h"
#include <string>

int run = 1;

void cleanup(int signum){
    run = 0;
}

int main(void){
    // create the shm server
    shm_server<std::string> *sherlock = new shm_server<std::string>(5678);
    // latch a cleanup function onto the SIGTERM signal
    ah_signal(SIGINT, &cleanup);

    while (run){
        sherlock->set("is a good guy!");
        sleep(1);
        sherlock->set("is a bad guy!");
        sleep(1);
        sherlock->set("is dead!");
        sleep(1);
        sherlock->set("is now alive again!");
        sleep(1);
    }
    
    delete sherlock;
    return 0;
}

Now, to connect to the memory.  We have to attach using the same key, which is a little bit of a paint, but if you use macros effectively, it'll just be like pointing at a file.  This client will be way faster than using files though, for time intensive processes, use these ``shm``s.

In [3]:
%%runcpp shm_client
#include "../ah_shm.h"
#include <string>

int main(void){
    shm_client<std::string> *holmes = new shm_client<std::string>(5678);
    for(int i=1; i<5; i++){
        printf("Episode %d in which Sherlock %s\n", i, holmes->get().c_str());
        sleep(1);
    }
    delete holmes;
    return 0;
}

Episode 1 in which Sherlock is now alive again!
Episode 2 in which Sherlock is a good guy!
Episode 3 in which Sherlock is a bad guy!
Episode 4 in which Sherlock is dead!



Now, we just have to stop the server to make sure we're not leaving too many things open.

In [4]:
%killall

Just a note:  I'm still working on the cleanup of this.  To check that I've removed all of the attached shared memory, you can use the bash line

```bash
ipcs
```

and to remove it, find the ``KEY`` corresponding to it and use
```bash
ipcrm -M <KEY>
```

In [7]:
!ipcs

IPC status from <running system> as of Tue May 23 22:24:49 EDT 2017
T     ID     KEY        MODE       OWNER    GROUP
Message Queues:

T     ID     KEY        MODE       OWNER    GROUP
Shared Memory:
m  65536 0x52010016 --rw-------     root    wheel

T     ID     KEY        MODE       OWNER    GROUP
Semaphores:



In [6]:
!ipcrm -M 0x0000162e